# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm 
import os
import urllib

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []
keys = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country_code = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code

    key = city+', '+country_code # create key for cities with the same name to use in API call (e.g. Athens GR vs Athens GA)

    # If the city is unique, then add it to a our cities list
    if key not in keys:
        cities.append(city)
        countries.append(country_code)
        keys.append(key)

# Print the city count to confirm sufficient count
print(len(keys))


623


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
weather = []
i=1

try: os.mkdir('output')
except: pass

log = os.path.join("output/log.txt")
with open(log, 'w') as textfile:
    
    settings = {"units": "metric", "appid": api_key}
    for key in keys:
        try:
            cityweather = owm.get_current(key, **settings)
            textfile.write(f"Processing record {i}: {cityweather['name']}, ID: {cityweather['id']} \n")        
        except urllib.request.HTTPError as e:
            if e.code == 429:
                textfile.write(f"Processing record {i}: {key} {e}, retrying in 61 sec\n")
                time.sleep(61)
                cityweather = owm.get_current(key, **settings)
            elif e.code == 404:
                textfile.write(f"{key} not found. Skipping City...\n")
                continue
            else:
                textfile.write(f"{e} Skipping {key}...\n")
                continue
        except BaseException as e:
                textfile.write(f"{e} Skipping {key}...\n")
                continue

        weather += [{'City':cityweather['name'],
                     'Country':cityweather['sys']['country'],
                     'City ID': cityweather['id'],
                     'Latitude':cityweather['coord']['lat'],
                     'Longditude':cityweather['coord']['lon'],
                     'Cloudiness':cityweather['clouds']['all'],
                     'Humidity':cityweather['main']['humidity'],
                     'Current Temperature':cityweather['main']['temp'],
                     'Max Temp':cityweather['main']['temp_max'],
                     'Min Temp':cityweather['main']['temp_min'],
                     # take avg to normalize for different time zones 
                     'Avg Temp':(cityweather['main']['temp_max']+cityweather['main']['temp_min'])/2,
                     'Wind Speed':cityweather['wind']['speed']
                    }]
        i += 1

with open(log, 'r') as textfile:                       
    print(textfile.read())
textfile.close()

Processing record 1: Port Hardy, ID: 6111862 
Processing record 2: Lebu, ID: 3883457 
Processing record 3: Camapua, ID: 3468020 
Processing record 4: Punta Arenas, ID: 3874787 
Processing record 5: Srednekolymsk, ID: 2121025 
Processing record 6: Carnarvon, ID: 2074865 
Processing record 7: Batagay, ID: 2027044 
Processing record 8: Torbay, ID: 6167817 
Processing record 9: Ponta do Sol, ID: 3374346 
Processing record 10: Saint-Philippe, ID: 935215 
mataura, pf not found. Skipping City...
Processing record 11: Guatire, ID: 3639898 
Processing record 12: Hermanus, ID: 3366880 
Processing record 13: Horta, ID: 2743292 
Processing record 14: Hithadhoo, ID: 1282256 
Processing record 15: Sao Geraldo do Araguaia, ID: 3388760 
samusu, ws not found. Skipping City...
Processing record 16: Severo-Kurilsk, ID: 2121385 
Processing record 17: Ushuaia, ID: 3833367 
Processing record 18: Riberalta, ID: 3906466 
Processing record 19: Norman Wells, ID: 6089245 
Processing record 20: Bluff, ID: 2206939

In [4]:
# Print new city count after lookup to confirm the count remains sufficient (to account for API calls not returned)
len(weather)

552

In [5]:
# Create DataFrame
weather_df = pd.DataFrame(weather)

# Change order of Columns
weather_df = weather_df[['City', 
                         'Country', 
                         'City ID',
                         'Latitude', 
                         'Longditude', 
                         'Cloudiness', 
                         'Humidity', 
                         'Current Temperature', 
                         'Max Temp', 
                         'Min Temp', 
                         'Avg Temp', 
                         'Wind Speed']]

# Export the city data into a .csv
weather_df.to_csv("output/weather.csv")

# Display the DataFrame
weather_df.head()

,City,Country,City ID,Latitude,Longditude,Cloudiness,Humidity,Current Temperature,Max Temp,Min Temp,Avg Temp,Wind Speed
0,Port Hardy,CA,6111862,50.70,-127.42,5,41,4.00,4.00,4.00,4.00,6.20
1,Lebu,CL,3883457,-37.62,-73.65,0,87,14.35,14.35,14.35,14.35,12.31
2,Camapua,BR,3468020,-19.53,-54.04,36,68,28.37,28.37,28.37,28.37,4.66
3,Punta Arenas,CL,3874787,-53.16,-70.91,90,87,9.65,10.00,9.00,9.50,8.70
4,Srednekolymsk,RU,2121025,67.46,153.71,48,33,-30.31,-30.31,-30.31,-30.31,1.16


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot